# 5 - Tokamak particle orbits

In this tutorial we will plot particle orbits in poloidal cross sections of Tokamaks.

## Vlasov vs. DriftKinetic

We shall consider the two models [Vlasov](https://struphy.pages.mpcdf.de/struphy/sections/models.html#struphy.models.toy.Vlasov) and [DriftKinetic](https://struphy.pages.mpcdf.de/struphy/sections/models.html#struphy.models.toy.DriftKinetic) next to each other.

In [ ]:
import os
import struphy

path_out = os.path.join(struphy.__path__[0], 'io/out/', 'tutorial_05a/')

os.listdir(path_out)

Let us get some meta data on the simulations used in this notebook:

In [ ]:
with open(os.path.join(path_out, 'meta.txt')) as file:
    print(file.read())

Let us look at the parameter file of the simulation:

In [ ]:
params_path = os.path.join(struphy.__path__[0], 'io/inp', 'tutorials', 'params_orbits_tokamak.yml')

import yaml

with open(params_path) as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)
    
parameters

Under the key word `kinetic` we see that only 4 particles were simulated, each one with carefully chosen initial conditions (`markers/loading/initial`).

The raw data has already been post-processed (presence of the folder `post_processing/`). Let us look at the resulting data:

In [ ]:
os.listdir(os.path.join(path_out, 'post_processing'))

Let us start by loading the time grid:

In [ ]:
import numpy as np

tgrid = np.load(os.path.join(path_out, 'post_processing', 't_grid.npy'))
Nt = len(tgrid) - 1
log_Nt = int(np.log10(Nt)) + 1

Nt

In [ ]:
os.listdir(os.path.join(path_out, 'post_processing', 'kinetic_data'))

In [ ]:
os.listdir(os.path.join(path_out, 'post_processing', 'kinetic_data', 'ions'))

In [ ]:
from tqdm import tqdm

Np = parameters['kinetic']['ions']['save_data']['n_markers']
print(Np)
    
pos = np.zeros((Nt + 1, Np, 3), dtype=float)

for n in tqdm(range(Nt + 1)):

    # load x, y, z coordinates
    orbits_path = os.path.join(path_out, 'post_processing', 'kinetic_data', 'ions', 'orbits', 'ions_{0:0{1}d}.txt'.format(n, log_Nt))
    pos[n] = np.loadtxt(orbits_path, delimiter=',')[:, 1:]

    # convert to R, y, z, coordinates
    pos[n, :, 0] = np.sqrt(pos[n, :, 0]**2 + pos[n, :, 1]**2)

At first, we want to plot the absolute value of the magnetic field in the poloidal plane.

We thus start by creating the `Domain` and `MHDequilibirum` objects from the parameter file:

In [ ]:
from struphy.io.setup import setup_domain_mhd

domain, mhd_equil = setup_domain_mhd(parameters)
domain_name = domain.__class__.__name__

The absolute value of $B$ at toroidal angle $\phi=0$ looks as follows (we create one plot for `Vlasov` and one plot for `DriftKinetic`):

In [ ]:
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(16, 8))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

e1 = np.linspace(0., 1., 101)
e2 = np.linspace(0., 1., 101)
e1[0] += 1e-5
X = domain(e1, e2, 0.)

im1 = ax1.contourf(X[0], X[2], mhd_equil.absB0(e1, e2, 0.), levels=51)
ax1.axis('equal')
ax1.set_title('abs(B) at $\phi=0$')
fig.colorbar(im1)

im2 = ax2.contourf(X[0], X[2], mhd_equil.absB0(e1, e2, 0.), levels=51)
ax2.axis('equal')
ax2.set_title('abs(B) at $\phi=0$')
fig.colorbar(im2)

We also need to build the grid; the grid info can be obtained as follows:

In [ ]:
import h5py

file = h5py.File(os.path.join(path_out, 'data/', 'data_proc0.hdf5'), 'r')
grid_info = file['scalar'].attrs['grid_info']
file.close()

grid_info

Let us now add the grid lines, as well as the plasma vessel wall (limiter):

In [ ]:
e1 = np.linspace(grid_info[0, 0], grid_info[0, 1],
                    int(grid_info[0, 2]) + 1)
e2 = np.linspace(grid_info[0, 3], grid_info[0, 4],
                    int(grid_info[0, 5]) + 1)
X = domain(e1, e2, 0.)

# plot xz-plane for torus mappings, xy-plane else
if 'Torus' in domain_name or domain_name == 'GVECunit' or domain_name == 'Tokamak':
    co1, co2 = 0, 2
else:
    co1, co2 = 0, 1

# eta1-isolines
for j in range(e1.size):
    if j == 0:
        pass
    elif j == e1.size - 1:
        ax1.plot(X[co1, j, :], X[co2, j, :], color='k')
        ax2.plot(X[co1, j, :], X[co2, j, :], color='k')
    else:
        ax1.plot(X[co1, j, :], X[co2, j, :], color='tab:blue', alpha=.25)
        ax2.plot(X[co1, j, :], X[co2, j, :], color='tab:blue', alpha=.25)

# # eta2-isolines
for k in range(e2.size):
    if k == 0:
        pass
    else:
        ax1.plot(X[co1, :, k], X[co2, :, k], color='tab:blue', alpha=.25)
        ax2.plot(X[co1, :, k], X[co2, :, k], color='tab:blue', alpha=.25)

ax1.plot(mhd_equil.limiter_pts_R, mhd_equil.limiter_pts_Z, 'tab:orange')
ax2.plot(mhd_equil.limiter_pts_R, mhd_equil.limiter_pts_Z, 'tab:orange')

ax1.set_xlabel('R [m]')
ax1.set_ylabel('Z [m]')
ax1.axis('equal')
ax1.set_title('abs(B) at $\phi=0$ with grid and limiter')

ax2.set_xlabel('R [m]')
ax2.set_ylabel('Z [m]')
ax2.axis('equal')
ax2.set_title('abs(B) at $\phi=0$ with grid and limiter')

fig

Finally, we add the particle orbits:

In [ ]:
for i in range(pos.shape[1]):
    ax1.scatter(pos[:, i, 0], pos[:, i, 2], s=1)
   
ax1.set_title('Passing and trapped particles (co- and counter direction)')
    
fig

We shall now add the [DriftKinetic](https://struphy.pages.mpcdf.de/struphy/sections/models.html#struphy.models.toy.DriftKinetic) particles:

In [ ]:
path_out = os.path.join(struphy.__path__[0], 'io/out/', 'tutorial_05b/')

os.listdir(path_out)

Let us get some meta data on the simulations used in this notebook:

In [ ]:
with open(os.path.join(path_out, 'meta.txt')) as file:
    print(file.read())

Let us look at the parameter file of the simulation:

In [ ]:
params_path = os.path.join(struphy.__path__[0], 'io/inp', 'tutorials', 'params_gc_orbits_tokamak.yml')

import yaml

with open(params_path) as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)
    
parameters

Under the key word `kinetic` we see that only 4 particles were simulated, each one with carefully chosen initial conditions (`markers/loading/initial`).

The raw data has already been post-processed (presence of the folder `post_processing/`). Let us look at the resulting data:

In [ ]:
os.listdir(os.path.join(path_out, 'post_processing'))

Let us start by loading the time grid:

In [ ]:
import numpy as np

tgrid = np.load(os.path.join(path_out, 'post_processing', 't_grid.npy'))
Nt = len(tgrid) - 1
log_Nt = int(np.log10(Nt)) + 1

print(Nt, log_Nt)

In [ ]:
os.listdir(os.path.join(path_out, 'post_processing', 'kinetic_data'))

In [ ]:
os.listdir(os.path.join(path_out, 'post_processing', 'kinetic_data', 'ions'))

In [ ]:
from tqdm import tqdm

Np = parameters['kinetic']['ions']['save_data']['n_markers']
    
pos = np.zeros((Nt + 1, Np, 3), dtype=float)

for n in tqdm(range(Nt + 1)):

    # load x, y, z coordinates
    orbits_path = os.path.join(path_out, 'post_processing', 'kinetic_data', 'ions', 'orbits', 'ions_{0:0{1}d}.txt'.format(n, log_Nt))
    pos[n] = np.loadtxt(orbits_path, delimiter=',')[:, 1:]

    # convert to R, y, z, coordinates
    pos[n, :, 0] = np.sqrt(pos[n, :, 0]**2 + pos[n, :, 1]**2)

Let us add the `DriftKinetic` orbits:

In [ ]:
for i in range(pos.shape[1]):
    ax2.scatter(pos[:, i, 0], pos[:, i, 2], s=1)
   
ax2.set_title('DriftKinetic version')
    
fig